In [19]:
import os
import json
import pandas as pd
import traceback

In [20]:
from dotenv import load_dotenv
load_dotenv()

True

In [21]:
key=os.getenv("OPENAI_API_KEY")

In [73]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.7,   
   
)

In [74]:
llm.invoke("tell me about django").content

'Django is a high-level web framework for Python that encourages rapid development and clean, pragmatic design. It was created to simplify the process of building web applications by providing a robust set of tools and features. Here are some key points about Django:\n\n1. **MVC Pattern**: Django follows the Model-View-Controller (MVC) design pattern, although it calls it Model-View-Template (MVT). This pattern helps in separating the business logic from the user interface.\n\n2. **ORM**: Django includes an Object-Relational Mapping (ORM) system which allows developers to interact with the database using Python code instead of writing raw SQL queries. This makes database interactions more intuitive and less error-prone.\n\n3. **Admin Interface**: One of Django\'s standout features is its automatically generated admin interface. After defining data models, Django can create a fully-featured, web-based administration panel without any additional coding.\n\n4. **Security**: Django comes w

In [75]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [76]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [77]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs

{response_json}

"""

In [78]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [79]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [80]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:"""

In [81]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)


In [82]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


In [83]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [84]:
file_path=r"C:\Users\shmujeeb\Desktop\HU\MCQ\data.txt"

In [85]:
file_path

'C:\\Users\\shmujeeb\\Desktop\\HU\\MCQ\\data.txt'

In [86]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [87]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [88]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [89]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [90]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [91]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1784
Prompt Tokens:1088
Completion Tokens:696
Total Cost:0.01588


In [92]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [93]:
quiz=response.get("quiz")

In [94]:
quiz=json.loads(quiz)

In [97]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [98]:
quiz=pd.DataFrame(quiz_table_data)

In [99]:
quiz_table_data

[{'MCQ': 'What is biology the scientific study of?',
  'Choices': 'a: Physics | b: Chemistry | c: Life | d: Astronomy',
  'Correct': 'Life'},
 {'MCQ': 'Which of the following is a unifying theme in biology?',
  'Choices': 'a: All organisms are made up of cells | b: Planets orbit the sun | c: Chemical reactions | d: Gravitational force',
  'Correct': 'All organisms are made up of cells'},
 {'MCQ': 'What major theme in biology explains the unity and diversity of life?',
  'Choices': 'a: Photosynthesis | b: Evolution | c: Magnetism | d: Gravity',
  'Correct': 'Evolution'},
 {'MCQ': 'Which process is important to life because it allows organisms to move, grow, and reproduce?',
  'Choices': 'a: Energy processing | b: Evaporation | c: Condensation | d: Erosion',
  'Correct': 'Energy processing'},
 {'MCQ': 'What do biologists use to make observations, pose questions, generate hypotheses, perform experiments, and form conclusions?',
  'Choices': 'a: Astrological charts | b: Scientific method |

In [100]:
quiz.to_csv("machinelearning.csv",index=False)